# Machine Learning

In [2]:
import pandas as pd

file = '../data/balanced_data.csv'

data = pd.read_csv(file)
data

,Unnamed: 0,index,Access Gained,Attack Origin,Authentication Required,Availability,CVE ID,CVE Page,CWE ID,Complexity,...,parentID,patch,project,project_after,project_before,target,vul_func_with_fix,processed_func,flaw_line,flaw_line_index
0,0,175663,None,Remote,Not required,Complete,CVE-2017-0596,https://www.cvedetails.com/cve/CVE-2017-0596/,NaN,Medium,...,NaN,"@@ -1110,7 +1110,9 @@\n\n if (inputBuf...",Android,https://android.googlesource.com/platform/fram...,https://android.googlesource.com/platform/fram...,0,OMX_ERRORTYPE SoftVPXEncoder::internalSetRoleP...,OMX_ERRORTYPE SoftVPXEncoder::internalSetRoleP...,NaN,NaN
1,1,153000,None,Remote,Not required,Partial,CVE-2016-5216,https://www.cvedetails.com/cve/CVE-2016-5216/,CWE-416,Medium,...,NaN,"@@ -2287,7 +2287,13 @@ int PDFiumEngine::GetMo...",Chrome,bf6a6765d44b09c64b8c75d749efb84742a250e7,d62b0ef3d51e6c96e3bb960c1a4fa395082dccf9,0,FPDF_BOOL PDFiumEngine::Form_PutRequestURL(FPD...,FPDF_BOOL PDFiumEngine::Form_PutRequestURL(FPD...,NaN,NaN
2,2,29046,None,Remote,Not required,Partial,CVE-2013-4623,https://www.cvedetails.com/cve/CVE-2013-4623/,CWE-20,Medium,...,NaN,"@@ -2375,8 +2375,8 @@ int ssl_parse_certificat...",polarssl,1922a4e6aade7b1d685af19d4d9339ddb5c02859,6417186365f4a73a719fff754fefe8edcef2bc28,0,static void ssl_update_checksum_md5sha1( ssl_c...,static void ssl_update_checksum_md5sha1( ssl_c...,NaN,NaN
3,3,59380,None,Local,Not required,Complete,CVE-2017-16939,https://www.cvedetails.com/cve/CVE-2017-16939/,CWE-416,Low,...,NaN,"@@ -1693,32 +1693,34 @@ static int dump_one_po...",linux,1137b5e2529a8f5ca8ee709288ecba3e68044df2,10a7ef33679073d13bf1dd05e3f1b7912f999543,0,static int xfrm_send_acquire(struct xfrm_state...,static int xfrm_send_acquire(struct xfrm_state...,NaN,NaN
4,4,106794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"@@ -401,12 +401,28 @@ void WebPage::gestureWil...",Chrome,df831400bcb63db4259b5858281b1727ba972a2a,ed03cbf291b69831f85347962d190a391c8b6bdc,0,void WebPage::setComposition(const String& com...,void WebPage::setComposition(const String& com...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21795,10895,186278,None,Remote,Not required,Partial,CVE-2017-5092,https://www.cvedetails.com/cve/CVE-2017-5092/,CWE-20,Medium,...,NaN,"@@ -148,24 +148,37 @@ bool BrowserPpapiHostImp...",Chrome,66b99f3fe60dce77f079cc9c07164f6a34dbea37,f1b5b08b61b3f3f23d2f6bd82339b20a7141603a,1,void BrowserPpapiHostImpl::DeleteInstance(PP_...,void BrowserPpapiHostImpl::DeleteInstance(PP_I...,DCHECK(it != instance_map_.end());/~/ // We...,"2,3,4,5,6,7,8,9,11"
21796,10896,180669,None,Remote,Not required,Complete,CVE-2015-8787,https://www.cvedetails.com/cve/CVE-2015-8787/,NaN,Low,...,NaN,"@@ -55,7 +55,7 @@ nf_nat_redirect_ipv4(struct ...",linux,94f9cd81436c85d8c3a318ba92e236ede73752fc,45efccdbec3cd465c4776ed9ca1d7b1bba1b7e34,1,"nf_nat_redirect_ipv4(struct sk_buff *skb,\n\t\...","nf_nat_redirect_ipv4(struct sk_buff *skb,\ncon...",\t\tif (indev != NULL) {,26
21797,10897,180610,None,Remote,Not required,Complete,CVE-2016-2070,https://www.cvedetails.com/cve/CVE-2016-2070/,CWE-189,Low,...,NaN,"@@ -2478,6 +2478,9 @@ static void tcp_cwnd_red...",linux,8b8a321ff72c785ed5e8b4cf6eda20b35d427390,58caf637365fef97c8e84ea5699a8e34d68fce93,1,static void tcp_cwnd_reduction(struct sock *sk...,static void tcp_cwnd_reduction(struct sock *sk...,NaN,NaN
21798,10898,180481,None,Remote,Not required,Partial,CVE-2016-4303,https://www.cvedetails.com/cve/CVE-2016-4303/,CWE-119,Low,...,NaN,"@@ -40,994 +40,723 @@\n #ifndef LLONG_MIN\n #d...",iperf,91f2fa59e8ed80dfbf400add0164ee0e508e412a,fec5bd1db87eed176c50b9697be2eb3e960b6661,1,static char *print_string( cJSON *item )\n//fl...,static char *print_string( cJSON *item )\n{\n\...,{/~/\treturn print_string_ptr( item->valuestri...,"1,2,3"


## Features

In [6]:
feature_columns = ['processed_func']
target_columns = ['target']

target = data[target_columns]
features = data[feature_columns]
features

,processed_func
0,OMX_ERRORTYPE SoftVPXEncoder::internalSetRoleP...
1,FPDF_BOOL PDFiumEngine::Form_PutRequestURL(FPD...
2,static void ssl_update_checksum_md5sha1( ssl_c...
3,static int xfrm_send_acquire(struct xfrm_state...
4,void WebPage::setComposition(const String& com...
...,...
21795,void BrowserPpapiHostImpl::DeleteInstance(PP_I...
21796,"nf_nat_redirect_ipv4(struct sk_buff *skb,\ncon..."
21797,static void tcp_cwnd_reduction(struct sock *sk...
21798,static char *print_string( cJSON *item )\n{\n\...


In [7]:
target

,target
0,0
1,0
2,0
3,0
4,0
...,...
21795,1
21796,1
21797,1
21798,1
